# Route Vizualization
### Huanlin Dai

Proof of concept: graphing routes

In [1]:
import pandas as pd
import folium
import geopandas
from geodatasets import get_path
import matplotlib
import contextily as cx
from shapely.geometry import LineString
import osmnx as ox
import networkx as nx

In [2]:
feu_galveston = pd.read_csv("../data/FUE_Galveston.csv")
route_1_data = pd.read_csv("../data/route1.csv")
route_2_data = pd.read_csv("../data/route2.csv")
coords1 = route_1_data[['Longitude', 'Latitude']]
coords2 = route_2_data[['Longitude', 'Latitude']]
route_1_data.head()

,Original_Index,Name,Weekly_Dropoff_Totes,Daily_Pickup_Totes,Address,Latitude,Longitude,Cumulative_Distance,Truck_Load
0,0,Moody Gardens,0.0,0.0,"1 Hope Blvd, Galveston",29.2736,-94.8523,0,0
1,116,Shearn's Seafood and Prime Steaks,1.0,1.0,"7 Hope Blvd Galveston, TX 77554",29.2708,-94.8505,471,1
2,22,Cajun Greek,1.0,1.0,"2226 61st St Galveston, TX 77551",29.2769,-94.8316,2969,2
3,150,Mario's Ristorante,1.0,1.0,"2202 61st St Galveston, TX 77551",29.2771,-94.8315,3049,3
4,144,Yamato,1.0,1.0,"2104 61st Street, Galveston",29.2781,-94.8316,3250,4


In [3]:
def find_bbox(coords):
    """
    Given a list of coordinates (longitude and latitude),
    find a bounding box that contains all the points of interest.
    This function helps reduce the number of nodes and edges in a 
    osmnx graph to reduce computational complexity/time.
    Parameters:
        coords : pd.DataFrame
    Returns:
        n, s, e, w : float
    """
    if len(coords) == 0:
        raise ValueError("find_bbox :: no coords inputted")
    n, s, e, w = [coords['Latitude'].iloc[0], coords['Latitude'].iloc[0],
                  coords['Longitude'].iloc[0], coords['Longitude'].iloc[0]]
    for i in range(len(coords)):
        longitude = coords["Longitude"].iloc[i]
        latitude = coords["Latitude"].iloc[i]
        n, s, e, w = [max(latitude, n), min(latitude, s),
                      max(longitude, e), min(longitude, w)]
    return n, s, e, w

In [4]:
n1, s1, e1, w1 = find_bbox(coords1)
n2, s2, e2, w2 = find_bbox(coords2)

In [5]:
place = "Galveston, Texas, USA"
galv_map = folium.Map(location = [29.30135, -94.7977], tiles ='OpenStreetMap', zoom_start=11)
galv_graph = ox.graph_from_place(place, network_type='drive')

In [6]:
galv_graph1 = ox.truncate.truncate_graph_bbox(galv_graph, n1, s1, e1, w1, truncate_by_edge=False, retain_all=False, quadrat_width=0.05, min_num=3)
galv_graph2 = ox.truncate.truncate_graph_bbox(galv_graph, n2, s2, e2, w2, truncate_by_edge=False, retain_all=False, quadrat_width=0.05, min_num=3)

In [7]:
def calc_routes(graph, coords):
    """
    Takes in a graph and a set of coordinates (w/ columns "Longitude" and "Latitude")
    and returns the set of shortest routes between each coordinate

    Parameters:
        graph : osmnx graph
        coords : dataframe
    Returns:
        routes: list of routes
        
    """
    routes = []
    for i in range(len(coords) - 1):
        start_node = ox.nearest_nodes(graph, coords.iloc[i]['Longitude'], coords.iloc[i]['Latitude'])
        end_node = ox.nearest_nodes(graph, coords.iloc[i+1]['Longitude'], coords.iloc[i+1]['Latitude'])
        routes.append(nx.shortest_path(graph, start_node, end_node, weight='length'))
    return routes

In [8]:
routes_1 = calc_routes(galv_graph1, coords1)
routes_2 = calc_routes(galv_graph2, coords2)

In [9]:
def osmnx_to_latlon(graph, routes):
    """
    given a route created by osmnx (node numbers), create a list of x, y coordinates
    to draw on folium 
    """
    final_route = []
    for route in routes:
        for point in route:
            final_route.append((graph.nodes[point]['y'], graph.nodes[point]['x']))
    return final_route

In [43]:
def add_markers(f_map, route_data, color):
    """
    given a folium map, route data (includes location names), and a color (str),
    draw markers on the given map

    Parameters:
        f_map : folium map
        route_data : pd.DataFrame
        color : str (e.g., "red", "blue")
    """
    
    icon_size= 100
    for i in range(len(route_data)):
        y, x = route_data[["Latitude", "Longitude"]].iloc[i]
        folium.Marker((y,x), popup=route_data['Name'][i], icon=folium.Icon(color=color)).add_to(f_map)
        icon_size = 40
    return None

In [25]:
route_1 = osmnx_to_latlon(galv_graph1, routes_1)
route_2 = osmnx_to_latlon(galv_graph2, routes_2)

In [46]:
galveston_map = folium.Map(location = [29.30135, -94.7977], tiles ='OpenStreetMap', zoom_start=11)
add_markers(galveston_map, route_1_data, 'blue')
add_markers(galveston_map, route_2_data, 'red')
folium.PolyLine(locations=route_2, color="red").add_to(galveston_map)
folium.PolyLine(locations=route_1, color="blue").add_to(galveston_map)
galveston_map.save("route.html")
galveston_map